In [ ]:

!pip install -U boto3 langchain langchain_aws -U -i https://pypi.tuna.tsinghua.edu.cn/simple

In [8]:
import boto3

In [76]:
from langchain_aws import BedrockLLM
from langchain_community.chat_models import BedrockChat
from langchain_community.llms.sagemaker_endpoint import LLMContentHandler
from langchain_community.llms import SagemakerEndpoint

In [15]:
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage,AIMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate


In [27]:
boto3_bedrock = boto3.client(
    service_name="bedrock-runtime",
    region_name='us-east-1'
)

In [82]:
llm_sonnet = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0",
                client = boto3_bedrock,
                  model_kwargs={"temperature": 0.1,
                                "top_k":10,
                                "max_tokens": 1024,
                                "top_p":0.9,
                                # "stop_sequences":['</standalone_question>','</output>']
                               },
                  streaming=False,callbacks=[StreamingStdOutCallbackHandler()])

In [112]:
llm_llama = ChatBedrock(model_id="meta.llama3-70b-instruct-v1:0",
                  model_kwargs={"temperature": 0.1,
                                "max_gen_len": 100,
                                "top_p":0.9,
                                # "stop_sequences":['</standalone_question>','</output>']
                               },
                  streaming=False,
                  callbacks=[StreamingStdOutCallbackHandler()]
                  )

In [190]:
llm_mistral = ChatBedrock(model_id="mistral.mistral-large-2402-v1:0",
                  model_kwargs={"temperature": 0.01,
                                "max_tokens": 100,
                                "top_p":0.9,
                                "stop":['</standalone_question>','</output>']
                               },
                  streaming=False,
                  callbacks=[StreamingStdOutCallbackHandler()]
                  )

In [138]:
llm_mix87 = ChatBedrock(model_id="mistral.mixtral-8x7b-instruct-v0:1",
                  model_kwargs={"temperature": 0.1,
                                "max_tokens": 100,
                                "top_p":0.9,
                                "stop":['</standalone_question>','</output>']
                               },
                  streaming=False,
                  callbacks=[StreamingStdOutCallbackHandler()]
                  )

In [191]:
prompt = "Given the following conversation in <conversation></conversation>, and a follow up user question in <question></question>.\\n<conversation>\\nuser: aws clean room是什么？\\nAI: AWS Clean Rooms是一种安全的多方计算服务,允许不同组织在不共享原始数据的情况下,对彼此的数据进行联合分析。它采用差分隐私技术,确保数据隐私和安全。与其他类似服务相比,AWS Clean Rooms无需将数据移动到其他平台,可直接在AWS S3上进行分析。它还与AWS Data Exchange集成,方便寻找数据合作伙伴。AWS Clean Rooms将取代亚马逊广告的AMC服务,为更广泛的行业提供数据合作解决方案。\\n</conversation>\\n\\n<question>\\n他有什么优势？\\n</question>\\n\\nplease use the context in the chat conversation to rephrase the user question to be a standalone question, respond in the original language of user's question, don't translate the chat history and user question.\\nif you don't understand the user's question, or the question is not relevant to the conversation. please keep the orginal question.\\nSkip the preamble, don't explain, go straight into the answer. Please put the standalone question in <standalone_question> tag\\n"

In [192]:
llm_mistral.invoke(input=prompt).content

' <standalone_question>AWS Clean Rooms有什么优势？'

In [128]:
llm_mix87.invoke(input=prompt)

 <standalone\_question>\ncould you tell me the advantages of AWS Clean Rooms?\n</standalone\_question>

AWS Clean Rooms 的优势在于：

1. 安全的多方计算服务：AWS Clean Rooms 允许不同组织在不共享原始数据的情况下,对彼此的数据进行联合分析。
2. 采

AIMessage(content=' <standalone\\_question>\\ncould you tell me the advantages of AWS Clean Rooms?\\n</standalone\\_question>\n\nAWS Clean Rooms 的优势在于：\n\n1. 安全的多方计算服务：AWS Clean Rooms 允许不同组织在不共享原始数据的情况下,对彼此的数据进行联合分析。\n2. 采', additional_kwargs={'usage': {}}, response_metadata={'model_id': 'mistral.mixtral-8x7b-instruct-v0:1'}, id='run-47e78939-481a-457a-b1c5-11b80ae8c8a4-0')

In [96]:
llm_llama.invoke(input=prompt)

 

<standalone_question>什么是AWS Clean Rooms的优势？</standalone_question>

它有多个优势。首先，它采用差分隐私技术，确保数据隐私和安全。其次，它无需将数据移动到其他平台，直接在AWS S3上进行分析。最后，它与AWS Data Exchange集成，方便寻找数据合作伙伴。

AIMessage(content=' \n\n<standalone_question>什么是AWS Clean Rooms的优势？</standalone_question>\n\n它有多个优势。首先，它采用差分隐私技术，确保数据隐私和安全。其次，它无需将数据移动到其他平台，直接在AWS S3上进行分析。最后，它与AWS Data Exchange集成，方便寻找数据合作伙伴。', additional_kwargs={'usage': {}}, response_metadata={'model_id': 'meta.llama3-70b-instruct-v1:0'}, id='run-771a0a9a-0343-414c-84fa-eba4a3781e3b-0')

In [65]:
def format_to_message(query:str, image_base64_list:list = None, role:str = "user"):
    '''
    history format:
    "history": [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": "iVBORw..."
                    }
                },
                {
                    "type": "text",
                    "text": "What's in these images?"
                }
            ]
        }
    ]
    '''
    if image_base64_list:
        content = [{ "type": "image", "source": { "type": "base64", "media_type": "image/jpeg", "data": image_base64 }} for image_base64 in image_base64_list ]
        content.append({ "type": "text", "text": query })
        return { "role": role, "content": content }

    return {"role": role, "content": query }

In [66]:
def create_detect_prompt_templete():   
    prompt_template = """Here is a list of aimed functions:
    <api_schemas>{api_schemas}</api_schemas>
    You should follow below examples to choose the corresponding function and params according to user's query
    <examples>{examples}</examples>
    """

    PROMPT = PromptTemplate(
        template=prompt_template, 
        input_variables=['api_schemas','examples']
    )
    return PROMPT

In [67]:
api_schema_options = ["""
{
        "name": "ec2_price",
        "description": "query the price of AWS ec2 instance",
        "parameters": {
            "type": "object",
            "properties": {
                "instance_type": {
                    "type": "string",
                    "description": "the AWS ec2 instance type, for example, c5.xlarge, m5.large, t3.mirco, g4dn.2xlarge, if it is a partial of the instance type, you should try to auto complete it. for example, if it is r6g.2x, you can complete it as r6g.2xlarge"
                },
                "region": {
                    "type": "string",
                    "description": "the AWS region name where the ec2 is located in, for example us-east-1, us-west-1, if it is common words such as 'us east 1','美东1','美西2',you should try to normalize it to standard AWS region name, for example, 'us east 1' is normalized to 'us-east-1', '美东2' is normalized to 'us-east-2','美西2' is normalized to 'us-west-2','北京' is normalized to 'cn-north-1', '宁夏' is normalized to 'cn-northwest-1', '中国区' is normalized to 'cn-north-1'"
                },
                "os": {
                    "type": "string",
                    "description": "the operating system of ec2 instance, the valid value should be 'Linux' or 'Windows'"
                },
                "term": {
                    "type": "string",
                    "description": "the payment term, 'RI' is normalized to 'Reserved'. the valid value should be 'OnDemand' or 'Reserved' "
                },
                "purchase_option": {
                    "type": "string",
                    "description": "the purchase option of Reserved instance, the valid value should be 'No Upfront', 'Partial Upfront' or 'All Upfront' "
                }
            },
            "required": ["instance_type"]
        }
    }
"""]

In [68]:
api_schema_str = "<api_schema>\n{}\n</api_schema>".format(",\n".join(api_schema_options))



In [105]:
example_list_str = """\\n<query>g4dn的价格是多少</query>\\n<output>\\"{\\\\\\"func\\\\\\": \\\\\\"ec2_price\\\\\\", \\\\\\"param\\\\\\": {\\\\\\"instance_type\\\\\\": \\\\\\"g4dn\\\\\\"}}\\"</output>\\n<query>c5.xlarge的windows实例价格？</query>\\n<output>\\"{\\\\\\"func\\\\\\": \\\\\\"ec2_price\\\\\\", \\\\\\"param\\\\\\": {\\\\\\"instance_type\\\\\\": \\\\\\"c5.xlarge\\\\\\", \\\\\\"os\\\\\\": \\\\\\"Windows\\\\\\"}}\\"</output>\\n<query>SageMaker中ml.t3.large的什么价格？</query>\\n<output>\\"{\\\\\\"func\\\\\\": \\\\\\"ec2_price\\\\\\", \\\\\\"param\\\\\\": {\\\\\\"instance_type\\\\\\": \\\\\\"ml.t3.large\\\\\\"}}\\"</output>\\n<query>t3.mirco在美西2的Linux实例什么价格？</query>\\n<output>\\"{\\\\\\"func\\\\\\": \\\\\\"ec2_price\\\\\\", \\\\\\"param\\\\\\": {\\\\\\"instance_type\\\\\\": \\\\\\"t3.mirco\\\\\\", \\\\\\"region\\\\\\": \\\\\\"us-west-2\\\\\\", \\\\\\"os\\\\\\": \\\\\\"Linux\\\\\\"}}\\"</output>\\n"""

In [201]:
example_list_str = """<query>m5.2x在美西2的价格？</query>\\n<output>\\"{\\\\\\"func\\\\\\": \\\\\\"ec2_price\\\\\\", \\\\\\"param\\\\\\": {\\\\\\"instance_type\\\\\\": \\\\\\"m5.2xlarge\\\\\\", \\\\\\"region\\\\\\": \\\\\\"us-west-2\\\\\\"}}\\"</output>\\n<query>g4dn.12x在美西2的价格？</query>\\n<output>\\"{\\\\\\"func\\\\\\": \\\\\\"ec2_price\\\\\\", \\\\\\"param\\\\\\": {\\\\\\"instance_type\\\\\\": \\\\\\"g4dn.12xlarge\\\\\\", \\\\\\"region\\\\\\": \\\\\\"us-west-2\\\\\\"}}\\"</output>\\n<query>g4dn在美西2的价格？</query>\\n<output>\\"{\\\\\\"func\\\\\\": \\\\\\"ec2_price\\\\\\", \\\\\\"param\\\\\\": {\\\\\\"instance_type\\\\\\": \\\\\\"g4dn\\\\\\", \\\\\\"region\\\\\\": \\\\\\"us-west-2\\\\\\"}}\\"</output>\\n<query>c5.xlarge在美东1的价格？</query>\\n<output>\\"{\\\\\\"func\\\\\\": \\\\\\"ec2_price\\\\\\", \\\\\\"param\\\\\\": {\\\\\\"instance_type\\\\\\": \\\\\\"c5.xlarge\\\\\\", \\\\\\"region\\\\\\": \\\\\\"us-east-1\\\\\\"}}\\"</output>\\n<query>c5.2xlarge的OnDemand什么价格？</query>\\n<output>\\"{\\\\\\"func\\\\\\": \\\\\\"ec2_price\\\\\\", \\\\\\"param\\\\\\": {\\\\\\"instance_type\\\\\\": \\\\\\"c5.x2large\\\\\\", \\\\\\"term\\\\\\": \\\\\\"OnDemand\\\\\\"}}\\"</output>"""

In [202]:
query = "g5.2x 在美东 1 区的价格是多少？"
# query = "bedrock限流怎么办"
prefix = """{"func":"""
prefill = """<query>{query}</query>\n<output>{prefix}""".format(query=query, prefix=prefix)

In [203]:
prompt_template = create_detect_prompt_templete()
prompt = prompt_template.format(api_schemas=api_schema_str, examples=example_list_str)
msg = format_to_message(query=prompt)
msg_list = [msg, {"role":"assistant", "content": prefill}]
msg_list

[{'role': 'user',
  'content': 'Here is a list of aimed functions:\n    <api_schemas><api_schema>\n\n{\n        "name": "ec2_price",\n        "description": "query the price of AWS ec2 instance",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "instance_type": {\n                    "type": "string",\n                    "description": "the AWS ec2 instance type, for example, c5.xlarge, m5.large, t3.mirco, g4dn.2xlarge, if it is a partial of the instance type, you should try to auto complete it. for example, if it is r6g.2x, you can complete it as r6g.2xlarge"\n                },\n                "region": {\n                    "type": "string",\n                    "description": "the AWS region name where the ec2 is located in, for example us-east-1, us-west-1, if it is common words such as \'us east 1\',\'美东1\',\'美西2\',you should try to normalize it to standard AWS region name, for example, \'us east 1\' is normalized to \'us-ea

In [204]:
prefix="""{"func":"""

In [205]:
answer = prefix+ llm_sonnet.invoke(msg_list).content
answer = answer.replace('<output>', '')
answer


  "ec2_price", "param": {"instance_type": "g5.2xlarge", "region": "us-east-1"}}

'{"func":  "ec2_price", "param": {"instance_type": "g5.2xlarge", "region": "us-east-1"}}'

In [206]:
import json
json.loads(answer)

{'func': 'ec2_price',
 'param': {'instance_type': 'g5.2xlarge', 'region': 'us-east-1'}}

In [211]:
answer = prefix+ llm_mistral.invoke(msg_list).content
answer = answer.replace('<output>', '')

In [212]:
answer

'{"func": "ec2_price", "param": {"instance_type": "g5.2xlarge", "region": "us-east-1"}}'

In [209]:
json.loads(answer)

{'func': 'ec2_price',
 'param': {'instance_type': 'g5.2xlarge', 'region': 'us-east-1'}}

In [179]:
import json
import boto3

SMM_KEY_AVAIL_LLM_ENDPOINTS = 'avail_llm_endpoints'
OTHER_ACCOUNT_LLM_ENDPOINTS = None

def get_all_bedrock_llm():
    model_ids = []
    try:
        bedrock = boto3.client(
            service_name='bedrock',
            region_name='us-west-2'
        )

        bedrock.list_foundation_models()

        response = bedrock.list_foundation_models(
            byOutputModality='TEXT',
            byInferenceType='ON_DEMAND'
        )
        model_ids = [ item['modelId'] for item in response['modelSummaries'] if not item['modelId'].startswith('mistral') ]
    except Exception as e:
        print(f"{str(e)}")

    return model_ids

In [180]:
get_all_bedrock_llm()

An error occurred (AccessDeniedException) when calling the ListFoundationModels operation: User: arn:aws:iam::434444145045:user/claude3_exp is not authorized to perform: bedrock:ListFoundationModels because no identity-based policy allows the bedrock:ListFoundationModels action


[]

In [38]:
template = """
You are a helpful assistant
Please answer users' question as possible as you can:
User question: {question}
"""


In [16]:
output_parser = StrOutputParser()
prompt_template = ChatPromptTemplate.from_template(template)

In [17]:
chain = prompt_template | llm_sonnet |output_parser

In [18]:
question = "who is the director of Terminator 2?"

In [19]:
chain.invoke({"question":question})

The director of Terminator 2: Judgment Day is James Cameron.

Some additional details:

- Terminator 2 was released in 1991 and was a sequel to the 1984 film The Terminator, which was also directed by James Cameron.

- It starred Arnold Schwarzenegger as the Terminator character, with Linda Hamilton and Edward Furlong in the other main roles.

- The film was a massive critical and commercial success, earning over $500 million at the box office against a $100 million budget.

- It was groundbreaking for its special effects, which still hold up incredibly well today thanks to the pioneering use of computer-generated imagery (CGI).

- James Cameron co-wrote the screenplay along with William Wisher Jr. He is considered a master of the sci-fi and action genres.

- Other famous films directed by James Cameron include Aliens, Titanic, and Avatar.

So in summary, the acclaimed director who brought the iconic Terminator 2: Judgment Day to life was the visionary filmmaker James Cameron.

'The director of Terminator 2: Judgment Day is James Cameron.\n\nSome additional details:\n\n- Terminator 2 was released in 1991 and was a sequel to the 1984 film The Terminator, which was also directed by James Cameron.\n\n- It starred Arnold Schwarzenegger as the Terminator character, with Linda Hamilton and Edward Furlong in the other main roles.\n\n- The film was a massive critical and commercial success, earning over $500 million at the box office against a $100 million budget.\n\n- It was groundbreaking for its special effects, which still hold up incredibly well today thanks to the pioneering use of computer-generated imagery (CGI).\n\n- James Cameron co-wrote the screenplay along with William Wisher Jr. He is considered a master of the sci-fi and action genres.\n\n- Other famous films directed by James Cameron include Aliens, Titanic, and Avatar.\n\nSo in summary, the acclaimed director who brought the iconic Terminator 2: Judgment Day to life was the visionary filmmaker James 